In [1]:
include("../src/modules/TUM69.jl")
include("../src/modules/Preprocess.jl")
include("../src/modules/Network.jl")
include("../src/modules/Utils.jl")

## Third party packages
using Knet: KnetArray, adam, relu, minibatch
using AutoGrad, Knet, CUDA


## Handwritten modules
using .TUM69: load_accel_data   # Data reading
using .Preprocess: process_accel_signal # Preprocessing on the data
using .Network: GCN, train_summarize!, accuracy4, nll4, GenericMLP # Construction of custom network
using .Utils: notify

In [2]:
function LR_norm(x; atype = Array{Float32}, o...)
    _, _, _, batch_size = size(x)
         
    x_ = []
            for k in 1:batch_size
                
               push!(x_, _LR_norm(x[: ,:, :, k]; atype = atype, o...))
   
            end

     x = cat(x_...; dims = 4)
        return x
     end

    function _LR_norm(x; atype =  Array{Float32}, o...)
    
        nx, ny, nc = size(x)
        
        x = mat(x; dims = 2)
        x_ = []#Array{atype}(undef, 0)
        for k = 1:(nx * ny)
            
        push!(x_, __LR_norm(x[k, :]; atype = atype, o...))
        
        end
    x_ = cat(x_...; dims = 2)
    x = reshape(x_', (nx, ny, nc))
    return x
        
    end

    function __LR_norm(x; atype =  Array{Float32}, k = 2, n = 5, alpha = 0.0001, beta = 0.75)
        nc = size(x, 1)
        x_ = [] #atype(undef, 0)
        for i in 1:nc
            _lower = convert(Int, floor(max(1., i - n/2)))
            _upper = convert(Int, floor(min(nc, i + n/2)))
            _sum = sum(x[_lower:_upper].^2)
            push!(x_, x[i] ./ ((k .+ alpha .* _sum).^beta))
        
        end
        x = x_
        return x
    end


__LR_norm (generic function with 1 method)

In [3]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

3×3×4×1 Array{Any,4}:
[:, :, 1, 1] =
 0.5      0.25      0.3
 0.2      0.166667  0.15
 0.12069  0.1       0.1

[:, :, 2, 1] =
 0.166667   0.222222   0.0714286
 0.0689655  0.136364   0.0408163
 0.0405405  0.0449438  0.0283019

[:, :, 3, 1] =
 0.0952381  0.111111  0.333333
 0.0789474  0.117647  0.214286
 0.137931   0.103448  0.097561

[:, :, 4, 1] =
 0.2       0.4       0.2
 0.147059  0.25      0.1
 0.1       0.153846  0.125

In [4]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)

Sparse(Array{Float32,4}(2,2,2,2)(2, 2, 2)(2, 2, 2))

In [5]:

function LR_norm(x; atype = Array, o...)
    
        _, _, _, batch_size = size(x)
        @show typeof(x)
        #if atype <: Array
            for k in 1:batch_size
                
                x[:, :, :, k] = mapslices(x -> _LR_norm(x; o...), (x[:, :, :, k]), dims = 3)
                
            end
       #= else
            for k in 1:batch_size
                
                x[:, :, :, k] = atype(mapslices(x -> _LR_norm(x; o...), Array(x[:, :, :, k]), dims = 3))
                
            end
        end
=#
        return x
     end



    function _LR_norm(x; k = 2, n = 5, alpha = 0.0001, beta = 0.75, atype = Array)
    
        nc = length(x)
        x_ = zeros(nc)
        #x_ = atype(zeros(nc))
        for i in 1:nc
            
            _lower = convert(Int, floor(max(1., i - n/2)))
            _upper = convert(Int, floor(min(nc, i + n/2)))
            _sum = sum(x[_lower:_upper].^2)
            x_[i] = x[i] ./ ((k .+ alpha .* _sum).^beta)
        end
        
        return x_
    end


_LR_norm (generic function with 1 method)

In [6]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

typeof(x) = Param{Array{Float32,4}}


3×3×4×1 Param{Array{Float32,4}}:
[:, :, 1, 1] =
 0.5      0.25      0.3
 0.2      0.166667  0.15
 0.12069  0.1       0.1

[:, :, 2, 1] =
 0.166667   0.222222   0.0714286
 0.0689655  0.136364   0.0408163
 0.0405405  0.0449438  0.0283019

[:, :, 3, 1] =
 0.0952381  0.111111  0.333333
 0.0789474  0.117647  0.214286
 0.137931   0.103448  0.097561

[:, :, 4, 1] =
 0.2       0.4       0.2
 0.147059  0.25      0.1
 0.1       0.153846  0.125

In [7]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)

typeof(x) = Param{Array{Float32,4}}

Stacktrace:
 [1] LR_norm(::Param{Array{Float32,4}}; atype::Type{T} where T, o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at .\In[5]:9
 [2] LR_norm(::Param{Array{Float32,4}}) at .\In[5]:4
 [3] (::var"#10#11")() at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:205
 [4] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:144
 [5] differentiate(::Function) at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:135
 [6] top-level scope at In[7]:3
 [7] include_string(::Function, ::Module, ::String, ::String) at .\loading.jl:1091
 [8] softscope_include_string(::Module, ::String, ::String) at C:\Users\volkan\.julia\packages\SoftGlobalScope\u4UzH\src\SoftGlobalScope.jl:65
 [9] execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\volkan\.julia\packages\IJulia\IDNmS\src\execute_request.jl:67
 [10] #invo

LoadError: MethodError: no method matching mapslices(::var"#7#8"{Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}}, ::AutoGrad.Result{Array{Float32,3}}; dims=3)
Closest candidates are:
  mapslices(::Any, !Matched::AbstractArray; dims) at abstractarray.jl:2061

In [210]:

function LR_norm_ground(x; atype = Array, o...)
    
        nx, ny, _, batch_size = size(x)
            for k in 1:batch_size
                for m in 1:nx
                for n in 1:ny
                
                x[m, n, :, k] = _LR_norm(x[m, n, :, k]; o...)
            end
            end
        end
        
    return x
end

    function _LR_norm(x; k = 2, n = 5, alpha = 0.0001, beta = 0.75, atype = Array)
        nc = length(x)
        x_ = zeros(nc)
        #x_ = atype(zeros(nc))
        for i in 1:nc
            
            _lower = convert(Int, floor(max(1., i - n/2)))
            _upper = convert(Int, floor(min(nc, i + n/2)))
            _sum = sum(x[_lower:_upper].^2)
            x_[i] = x[i] ./ ((k .+ alpha .* _sum).^beta)
        end
        x = x_
        return x
    end


_LR_norm (generic function with 1 method)

In [9]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

3×3×4×1 Param{Array{Float32,4}}:
[:, :, 1, 1] =
 0.5      0.25      0.3
 0.2      0.166667  0.15
 0.12069  0.1       0.1

[:, :, 2, 1] =
 0.166667   0.222222   0.0714286
 0.0689655  0.136364   0.0408163
 0.0405405  0.0449438  0.0283019

[:, :, 3, 1] =
 0.0952381  0.111111  0.333333
 0.0789474  0.117647  0.214286
 0.137931   0.103448  0.097561

[:, :, 4, 1] =
 0.2       0.4       0.2
 0.147059  0.25      0.1
 0.1       0.153846  0.125

In [10]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)


Stacktrace:
 [1] setindex!(::Array{Float64,1}, ::AutoGrad.Result{Float64}, ::Int64) at .\array.jl:847
 [2] _LR_norm(::AutoGrad.Result{Array{Float32,1}}; k::Int64, n::Int64, alpha::Float64, beta::Float64, atype::Type{T} where T) at .\In[8]:26
 [3] _LR_norm(::AutoGrad.Result{Array{Float32,1}}) at .\In[8]:18
 [4] LR_norm(::Param{Array{Float32,4}}; atype::Type{T} where T, o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at .\In[8]:9
 [5] LR_norm(::Param{Array{Float32,4}}) at .\In[8]:4
 [6] (::var"#14#15")() at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:205
 [7] differentiate(::Function; o::Base.Iterators.Pairs{Union{},Union{},Tuple{},NamedTuple{(),Tuple{}}}) at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:144
 [8] differentiate(::Function) at C:\Users\volkan\.julia\packages\AutoGrad\VFrAv\src\core.jl:135
 [9] top-level scope at In[10]:3
 [10] include_string(::Function, ::Module, ::String, ::String) at .\loading.jl:1091
 [11] softscope_

LoadError: MethodError: Cannot `convert` an object of type AutoGrad.Result{Float64} to an object of type Float64
Closest candidates are:
  convert(::Type{T}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S} at C:\Users\volkan\.julia\packages\Ratios\uRs4y\src\Ratios.jl:13
  convert(::Type{T}, !Matched::T) where T<:Number at number.jl:6
  convert(::Type{T}, !Matched::Number) where T<:Number at number.jl:7
  ...

In [144]:

function LR_norm(x::T; o...) where T
    
        nx, ny, nc, batch_size = size(x)
        
        x = mat(permutedims(x, (3,1,2,4)); dims = 1)

        x = x'
        
        y = similar(x)
    
        x = getindex.([x], 1:size(x, 1), :)
        @show x
        @show size(x)
        y = _LR_norm.(x; o...)

        y = vcat(y'...)

        y = reshape(y, (nx, ny, batch_size, nc))

        y = permutedims(y, (1,2,4,3))

    return y
end

    function _LR_norm(x::T; k = 2, n = 5, alpha = 0.0001, beta = 0.75) where T
 
    #@show size(x)
    k, n, alpha, beta = convert.(eltype(T), [k, n, alpha, beta]) 
    
    nc = length(x)
    
    _sum = []
    
        for i in 1:nc
            
            _lower = convert(Int, floor(max(1., i - n/2)))
            _upper = convert(Int, floor(min(nc, i + n/2)))
            push!(_sum,  sum(x[_lower:_upper].^2))
            
        end
    _sum = vcat(_sum...)
        return x./((k .+ alpha .* _sum).^beta)
    end

_LR_norm (generic function with 1 method)

In [143]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

x = Array{Float32,1}[[1.0, 1.0, 2.0, 4.0], [4.0, 2.0, 3.0, 5.0], [7.0, 3.0, 4.0, 2.0], [2.0, 2.0, 1.0, 2.0], [3.0, 3.0, 2.0, 2.0], [8.0, 4.0, 3.0, 2.0], [3.0, 1.0, 2.0, 1.0], [6.0, 2.0, 3.0, 1.0], [9.0, 3.0, 4.0, 4.0]]
size(x) = (9,)
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper = 4
_lower = 3
_upper = 4
_lower = 1
_upper = 2
_lower = 1
_upper = 3
_lower = 2
_upper 

3×3×4×1 Array{Float32,4}:
[:, :, 1, 1] =
 0.5      0.25      0.3
 0.2      0.166667  0.15
 0.12069  0.1       0.1

[:, :, 2, 1] =
 0.166667   0.222222   0.0714286
 0.0689655  0.136364   0.0408163
 0.0405405  0.0449438  0.0283019

[:, :, 3, 1] =
 0.0952381  0.111111  0.333333
 0.0789474  0.117647  0.214286
 0.137931   0.103448  0.097561

[:, :, 4, 1] =
 0.2       0.4       0.2
 0.147059  0.25      0.1
 0.1       0.153846  0.125

In [13]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)

2×2×2×2 Array{Float32,4}:
[:, :, 1, 1] =
  0.9419   0.828267
 -1.03967  0.209034

[:, :, 2, 1] =
 -0.62153   0.0951096
  0.547716  0.00498526

[:, :, 1, 2] =
 0.0671877  -0.868887
 2.07471     0.901451

[:, :, 2, 2] =
  0.0172826  0.980454
 -1.15379    0.0162631

In [14]:

function LR_norm(x::T; o...) where T
    
        nx, ny, nc, batch_size = size(x)
        
        x = mat(permutedims(x, (3,1,2,4)); dims = 1)

        x = x'
        
        y = similar(x)
    
        x = getindex.([x], 1:size(x, 1), :)

        y = _LR_norm.(x; o...)

        y = vcat(y'...)

        y = reshape(y, (nx, ny, batch_size, nc))

        y = permutedims(y, (1,2,4,3))

    return y
end

    function _LR_norm(x::T; k = 2, n = 5, alpha = 0.0001, beta = 0.75 , el_type = Float32) where T
    
    nc = length(x)
    k, n, alpha, beta = convert.(el_type, [k, n, alpha, beta]) 
    
    kernel_size = convert(Int, n+1)
    l_padding = convert(Int, ceil(n/2))
    r_padding = convert(Int, floor(n/2))
    
    x_ = reshape(vcat(zeros(el_type, l_padding), x, zeros(el_type, r_padding)), (nc + l_padding + r_padding, 1, 1, 1))
    
    w = reshape(ones(el_type, kernel_size), (kernel_size, 1, 1, 1))
        
    _sum = mat(conv4(w, x_.^2); dims = 4)

    return x./((k .+ alpha .* _sum).^beta)
    end

_LR_norm (generic function with 1 method)

In [15]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

3×3×4×1 Array{Float32,4}:
[:, :, 1, 1] =
 0.5      0.25      0.3
 0.2      0.166667  0.15
 0.12069  0.1       0.1

[:, :, 2, 1] =
 0.166667   0.222222   0.0714286
 0.0689655  0.136364   0.0408163
 0.0405405  0.0449438  0.0283019

[:, :, 3, 1] =
 0.0952381  0.111111  0.333333
 0.0789474  0.117647  0.214286
 0.137931   0.103448  0.097561

[:, :, 4, 1] =
 0.2       0.4       0.2
 0.147059  0.25      0.1
 0.1       0.153846  0.125

In [16]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)

2×2×2×2 Array{Float32,4}:
[:, :, 1, 1] =
 0.161849   0.354331
 0.817444  -0.277698

[:, :, 2, 1] =
 0.270671   0.0163726
 0.221797  -0.804352

[:, :, 1, 2] =
  0.477411   0.486567
 -0.239793  -0.199782

[:, :, 2, 2] =
  0.651453  -0.376088
 -0.511495  -0.907096

In [145]:

function LR_norm(x::T; k = 2, n = 5, alpha = 0.0001, beta = 0.75 , el_type = Float32) where T
    
        nx, ny, nc, batch_size = size(x)
        k, n, alpha, beta = convert.(el_type, [k, n, alpha, beta]) 

        kernel_size = convert(Int, n+1)
        l_padding = convert(Int, ceil(n/2))
        r_padding = convert(Int, floor(n/2))

        x_ = cat(zeros((nx,ny,l_padding,batch_size)), x, dims = 3)
        x_ = cat(x_, zeros((nx,ny,r_padding,batch_size)), dims = 3)
        
        #w = reshape(ones(el_type, (nc + l_padding + r_padding) * nc ), (nc + l_padding + r_padding, 1, , nc))

        w = reshape(ones(el_type, nx * ny * kernel_size), (kernel_size, 1, ny, nx))
        #w = reshape(ones(el_type, nx * 1 * kernel_size), (kernel_size, 1, 1, nx))
        
        x_ = permutedims(x_, (3,1,2,4))
        #_sum = depthwiseconv(x_.^2, w)
        _sum = conv4(w, x_.^2)
     
        _sum = permutedims(_sum, (3,2,1,4))

    return x./((k .+ alpha .* _sum).^beta)
         

      
end



LR_norm (generic function with 1 method)

In [146]:
x = Array{Float32}(randn(3,3,4,1))
x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y = LR_norm(x; k = 0, n = 2, alpha = 1, beta = 1)

3×3×4×1 Array{Float64,4}:
[:, :, 1, 1] =
 0.05  0.025641   0.0131579
 0.2   0.0384615  0.0263158
 0.35  0.102564   0.0394737

[:, :, 2, 1] =
 0.0344828  0.02  0.00371747
 0.0689655  0.03  0.00743494
 0.103448   0.04  0.0111524

[:, :, 3, 1] =
 0.0555556  0.0144928  0.020202
 0.0833333  0.0289855  0.030303
 0.111111   0.0434783  0.040404

[:, :, 4, 1] =
 0.133333   0.0384615  0.0153846
 0.166667   0.0384615  0.0153846
 0.0666667  0.0384615  0.0615385

In [19]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x))
grad(J, x)

┌ Warning: Slow fallback implementation invoked for ∇conv_data!  You probably don't want this; check your datatypes.
│   yT = Float64
│   T1 = Float64
│   T2 = Float32
└ @ NNlib C:\Users\volkan\.julia\packages\NNlib\2Wxlq\src\conv.jl:206


2×2×2×2 Array{Float64,4}:
[:, :, 1, 1] =
 0.115215  0.109449
 0.359937  0.608754

[:, :, 2, 1] =
 -0.666619    -0.183724
 -0.00813191   0.309822

[:, :, 1, 2] =
 -0.228018  0.596748
 -0.770403  0.139626

[:, :, 2, 2] =
 1.18588   -0.338958
 0.450504   1.18308

In [252]:
function LR_norm(x::T; k = 2, n = 5, alpha = 0.0001, beta = 0.75 , el_type = Float32) where T
            k, alpha, beta = convert.(el_type, [k, alpha, beta]) 

           nx, ny, nc, batch_size = size(x)
           x = permutedims(x, (3,1,2,4))
           x = reshape(x, (nc, nx * ny, 1, batch_size))
           kernel_size = convert(Int, n+1)
           #l_padding = convert(Int, ceil(n/2))
           #r_padding = convert(Int, floor(n/2))
           w = reshape(ones(el_type, kernel_size), (kernel_size, 1, 1, 1))

           _sum = conv4(w, x.^2; padding=(convert(Int, ceil(n/2)), 0))
            @show _sum
           println(size(_sum))
            println(divrem(n, 2)[2])
    _sum = _sum[1:(end - divrem(n, 2)[2]), :, :, :]
    
    println(size(_sum))

           y = x./((k .+ alpha .* _sum).^beta)
           println(size(y))
           y = reshape(y, (nc, nx, ny, batch_size))
           y = permutedims(y, (2,3,1,4))
           return y
 end

LR_norm (generic function with 1 method)

In [253]:
x = Array{Float32}(randn(3,3,12,1))
#x = Param(x)
x[:, :, 1, 1] = [1  2  3; 4  3  6; 7  8  9]
x[:, :, 2, 1] = [1  2  1; 2  3  2; 3  4  3]
x[:, :, 3, 1] = [2  1  2; 3  2  3; 4  3  4]
x[:, :, 4, 1] = [4  2  1; 5  2  1; 2  2  4]
y1 = LR_norm(x; k = 0, n = 5, alpha = 1, beta = 1)


_sum = Float32[6.0 29.0 74.0 9.0 22.0 89.0 14.0 49.0 106.0; 22.0 54.0 78.0 13.0 26.0 93.0 15.0 50.0 122.0; 22.016237 54.00505 79.347855 13.336161 26.02192 93.02641 15.288069 52.83959 122.20041; 22.229982 55.000565 79.36575 13.475053 26.16474 93.65268 15.862909 52.95526 124.34024; 22.178411 39.807907 30.392681 9.833733 17.537857 29.675438 9.737771 17.439756 44.318428; 22.12865 37.43903 21.443958 6.0154915 8.655146 14.096523 8.756595 13.572687 37.579597; 18.702986 28.450703 5.745508 5.716008 6.463887 5.152652 5.743382 7.4537773 22.153341; 2.717424 3.4509714 1.7613361 1.9070263 3.1714578 3.3805761 8.9846115 6.5367146 6.204432; 2.9233763 4.6582108 0.47381315 3.9295945 6.2745457 3.3575506 8.987156 6.043023 6.0976963; 3.480628 3.6626978 0.567831 3.81488 8.644521 4.544375 9.20323 5.935962 4.0187163; 2.5321999 2.8553524 0.5409064 3.4562006 8.271402 4.521616 6.3283687 5.4514685 3.0405247; 1.5819603 1.2242312 0.48962772 3.2744417 8.154115 4.10053 6.3095446 5.318537 0.77935547; 1.0076264 1.212559

3×3×12×1 Array{Float32,4}:
[:, :, 1, 1] =
 0.166667   0.222222   0.214286
 0.137931   0.136364   0.122449
 0.0945946  0.0898876  0.0849057

[:, :, 2, 1] =
 0.0454545  0.153846   0.0666667
 0.037037   0.115385   0.04
 0.0384615  0.0430108  0.0245902

[:, :, 3, 1] =
 0.090842   0.0749841  0.130821
 0.0555504  0.0768583  0.0567756
 0.0504109  0.0322489  0.0327331

...

[:, :, 10, 1] =
 0.0345232  -0.114566  0.223772
 0.0044788   0.097307  0.0485159
 0.221562    0.328455  0.0562451

[:, :, 11, 1] =
 -0.18615   -0.444365   -0.0851854
  0.385606  -0.213721    0.280958
 -0.454116  -0.0128778   0.100662

[:, :, 12, 1] =
  0.555048    -0.0474866   0.140951
 -0.00091907  -0.194402   -0.0174512
  0.683248     0.328374    0.316506

In [254]:
x = Array{Float32}(randn(2,2,2,2))
x = Param(x)
J = @diff sum(abs2, LR_norm(x; n = 5))
grad(J, x)

_sum = R(Array{Float32,4}(3,4,1,2))
(3, 4, 1, 2)
1
(2, 4, 1, 2)
(2, 4, 1, 2)


2×2×2×2 Array{Float32,4}:
[:, :, 1, 1] =
  0.338469  -1.71724
 -0.771704  -1.3464

[:, :, 2, 1] =
 0.407922  0.135006
 0.501176  0.904699

[:, :, 1, 2] =
 -1.21041   -0.0561789
  0.260603   0.209304

[:, :, 2, 2] =
 -0.713168  1.06001
  0.770997  0.403031

In [20]:
#=
using Test
a = randn(50,300,120,10)
aa = permutedims(a, (3,1,2,4))
aaa = mat(aa; dims = 1)
aaa = permutedims(aaa, (2,1))
aaaa = reshape(aaa, (50,300,10,120))
aaaaa = permutedims(aaaa, (1,2,4,3))
@test aaaaa == a
=#